<a href="https://colab.research.google.com/github/nokomoro3/book-ml-transformers/blob/main/ml-transformers-chap03-transformer-anatomy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 入門Transformers

- TransformerとULMFiTの２つの登場がきっかけ
- これにより、Transformerと教師なし学習を組み合わせることで、BERTやGPTが生まれた

![](./img/ml-transformers-chap01-introduction_2022-08-28-12-56-11.png)

- 重要なポイントは以下
  - Encoder-Decoder
  - Attention
  - Transfer Learning

## 1.1 エンコーダ・デコーダのフレームワーク

- 最初はRNNを使ったEncoder-Decoderモデル
- Encoderの役割
  - 入力系列を最後の隠れ状態という数値表現にエンコードする
- Decoderの役割
  - エンコード結果から出力系列を生成する

![](./img/ml-transformers-chap01-introduction_2022-08-28-12-57-42.png)

- Encoder-Decoderの弱点
  - この最後の隠れ状態が情報のボトルネックとなる
  - 入力全体の意味を表す必要がある
  - しかし実際は、系列が長い場合は意味を表現するのが難しい
  - Attentionでは、DecoderがEncoderのすべての隠れ状態にアクセスすることでこのボトルネックを解消する

## 1.2 アテンション機構

- アテンションの仕組み
  - Encoderのすべての時刻の状態を使用する
  - Decoderの時刻に応じて異なる重みで、Encoderの各時刻の状態を混ぜる
  - この重みをAttentionと呼ぶ

![](./img/ml-transformers-chap01-introduction_2022-08-28-13-00-18.png)

- Attentionの弱点
  - 逐次計算が必要なため入力系列全体で並列化できない

- Self-Attention(Transformer)の特徴
  - RNNによる再帰性を完全に排除
  - Encoder、Decoderのそれぞれの内に閉じたAttention機構を実装
  - これが『Attention is all you need』という論文

![](./img/ml-transformers-chap01-introduction_2022-08-28-13-00-36.png)

- Transformerの課題
  - 翻訳の大規模コーパスを用いてゼロから学習している
  - 実用的には、大量のラベル付きデータは使えないため、タスクが限定されてしまう。

## 1.3 NLPにおける転移学習

- 画像の場合
  - body（backbone）をImageNetで学習
  - headをタスクに特化したものとして、fine-tuningするのが一般的

- NLPの場合
  - 長年、NLPでどうすべきかという標準的なアプローチが明らかではなかった
  - 2017年～2018年でようやく明らかになってきた
  - きっかけはOpenAIの研究
    - [https://arxiv.org/abs/1704.01444](https://arxiv.org/abs/1704.01444)
  - 教師なしの事前学習から抽出した特徴を使用して、センチメント分類タスクで高性能を達成
  - その後、ULMFiTが登場し、事前学習済みのLSTMモデルを適応させるフレームワークを導入した
  - 関連研究としてELMoなどもある
    - ELMoでは、LSTMを次元学習し、下流タスクで使える高品質な単語埋め込みの生成が可能

- UMLFiTとは
  - 事前学習
      - 出現済みの単語から次の単語を予測するタスクを学習
      - 言語モデルとも呼ばれる
      - ラベル付きデータが不要なところがポイント
      - Wikipediaなどで学習
  - ドメイン適応
    - 例えば、IMDbコーパスに適応させる
    - その場合、IMDbのコーパスの次の単語予測を学習し、言語モデルを更新する
      - なので解く問題自体は事前学習と同じ
  - ファインチューニング
    - ターゲットタスク（IMDbのセンチメント分類）向けの分類層と言語モデルを同時に学習する
  - これが転移学習のフレームワークとして一般的に使用されることになる

- BERT, GPTへ
  - Transformerと転移学習を組み合わせたものとしてBERTとGPTがリリースされた

- GPT
  - Transformerのデコーダ部分のみを使用し、UMLFiTと同じ言語モデルアプローチを使用
  - 7000冊の未発表書籍からなるBookCorpusで事前学習されている

- BERT
  - Transformerのエンコーダ部分と、マスク言語モデルと呼ばれる特殊な言語モデルを使用
  - マスク言語モデル
      - テキスト中のランダムにマスクされた単語を予測する
  - BookCorpusと英語版wikipediaで事前学習されている

- 以降、これらの派生形のモデルが多く生まれたが、互換性のないフレームワークがつかわれていた。
- これをTransformersのリリースにより、50以上のアーキテクチャが統一されたAPIが構築
- これらが研究者に伝わり使用されるようになっている

## 1.4 入門Hugging Face Transformers

- Transformerモデルの標準化されたインターフェースを提供
- PyTorch、TensorFlow、JAXをサポートしており、簡単に切り替えが可能
- 更にタスクに特化したヘッドを提供しており、様々な下流タスクにtuningができる。
  - テキスト分類
  - 固有表現認識
  - 質問応答

## 1.5 Transformer の応用を巡る旅

- サンプル入力

In [ ]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure
from your online store in Germany. Unfortunately, when I opened the package,
I discovered to my horror that I had been sent an action figure of Megatron
instead! As a lifelong enemy of the Decepticons, I hope you can understand my
dilemma. To resolve the issue, I demand an exchange of Megatron for the
Optimus Prime figure I ordered. Enclosed are copies of my records concerning
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

### 1.5.1 テキスト分類

- pipelineを使い必要なステップを抽象化することができる
- `text-classification`パイプラインは、デフォルトではセンチメント分類用に設計されたモデルを使用するが、マルチクラス、マルチラベルの分類もサポートしている。

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification")

- ネガティブ・ポジティブを分類できる。

In [ ]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

### 1.5.2 固有表現認識

- 固有表現認識(named entity recognition:NER)
- aggregation_strategyは、複数単語をグルーピングする際のルールのようなもの。
- スコアはどの程度確信をもっていたかどうかを表します。

In [ ]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

- `##`はtokenizerが単語をトークンに分割した結果として表れる。
- start, endを使って`text[start:end]`とすることで単語を得ることもできる。

### 1.5.3 質問応答

- contextがtextとなり、それに対するquestionを与えると、answerを出力する。

In [ ]:
reader = pipeline("question-answering")

question = "What does the customer want?" # 顧客は何を求めていますか？
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

- この出力は、textから特定の部分を抽出しており、抽出型質問応答と呼ばれます。
- 質問応答には、抽出型以外にも様々な種類があります（詳細は第７章にて）

### 1.5.4 要約

- 今までのタスクより困難なタスク
- 矛盾しない文章の生成が必要
- summarizerにいくつかの引数を渡している

In [ ]:
summarizer = pipeline("summarization")

outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
print(outputs[0]["summary_text"])

### 1.5.5 翻訳

- 要約と同様に翻訳された文章を出力する
- Hugging Face Hubには数千言語のペアに対するもでるがある
- pipelineやtranslatorに引数を指定している

In [ ]:
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")

outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

### 1.5.6 テキスト生成

- 自動補完機能のようなもの

In [ ]:
generator = pipeline("text-generation")

response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response

outputs = generator(prompt, max_length=200)
print(outputs[0]['generated_text'])

## 1.6 Hugging Face のエコシステム

- 最初はTransformersのみだけでしたが、その後ライブラリやツールが追加されたエコシステム全体に成長している
- Hubで学習済みモデルやデータセット、評価指標のためのスクリプトを提供
- ライブラリは様々なコードを提供する

![](./img/ml-transformers-chap01-introduction_2022-08-28-13-14-48.png)

### 1.6.1 Hugging Face Hub

- 20,000以上のモデルを自由に利用可能
- 利用する場合、pipelineを使って利用する
- モデルやデータセットの内容について説明するカードがある
- 画面上でwidgetをつかって直接試すことも可能
- PyTorchやTensorFlowでも独自のHubが提供されており、Hugging Faceで利用できないものがある場合はそちらもチェックすることが推奨される

### 1.6.2 Hugging Face Tokenizers

- 生テキストをトークン化により分割するステップがある
- トークンは様々な単位がありうるが、単語、単語の一部や句読点などの文字と理解すればよい。
- Tokenizersは多数のトークン化戦略を提供しており、Rust実装により非常に高速な動作が可能。
- 入力の正規化やフォーマットの変換などの事前、事後処理も行うことができる。

### 1.6.3 Hugging Face Datasets

- データセットの読み込みは一般的に面倒なプロセスとなる
- 特にデータセットが大きい場合、非常に困難となる
- Datasetsを使うことで、標準的なインターフェースや、キャッシュ機能、メモリマッピングによるメモリ制限の回避なども機能として使用することが可能です。
- Datasetsは、PandasやNumPyとも相互運用が可能
- Datasetsはそのデータを評価するスクリプトも提供し、実験の再現性と結果の信頼性も高めます

### 1.6.4 Hugging Face Accelerate

- クラスタ上で学習を移植可能にするため、学習ループに抽象化されたレイヤを提供します
- Transformerをゼロから学習する必要がある場合などに必要となります。

## 1.7 Transformer の主な課題

- 言語
  - 多くの研究が英語を対象にしていおり、希少な言語の学習済みモデルを見つけることが簡単ではない。
  - 『４章：多言語の固有表現認識』では、多言語Transformerとそのゼロショットによる言語間遷移について検討する
- データの可用性
  - 転移学習を使うことでラベル付きデータの量を大幅に削減できるが、人間と比較するとまだまだ多くのデータが必要
  - 『９章：ラベルのないまたは少ない状況への対応方法』では、ラベルがほとんどないようなシナリオに取り組みます
- 長い文章の処理
  - Self-Attentionはparagraph単位のテキストではうまく機能しますが、文書全体のような長いテキストは計算量が非常に多くなる
  - 『１１章：Transformerの未来』では計算量を軽減するためのアプローチを説明します
- 透明性
  - 他のDeep Learningモデルと同様に、解釈性が低いことがTransformerでも欠点
  - 『２章：テキスト分類』と『４章：多言語の固有表現認識』でモデルのエラーを探る方法を何点か紹介します
- バイアス
  - Transformerはインターネット上のデータを使って事前学習されるため、バイアスがモデルに刷り込まれます。
  - この問題については『１０章：Transformerをゼロから学習する』で詳しく説明します

## 1.8 まとめ

- 本章ではわずかなコードで実行が可能でしたが、これはほんの一部
- 以降でより詳しく学んでいく